In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np
import pandas as pd

In [3]:
STORM_NUMBER = 8
TITLE = "16-17 February 2022"
SUBTITLE = "8 AM 18 February 2022"
sts = utc(2022, 2, 17, 12)
ets = utc(2022, 2, 18, 20, 0)
# Get available data
ctx = {'coop': 'yes', 't': 'state', 'sz': 60, 'z': 'yes', 'f': 'linear', 'v': 'snow', 'wfo': 'DMX'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, 'IA')
# add zeros and QC
df = add_zeros(df, ctx)

In [4]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2021-2022 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df2)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [14]:
cull = ['IA-ST-3', 'IA-DM-15']
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["val"] < 1].index, USEME] = False
hardcode = [
    ['IA-ST-48', 0.5],
    ['MZUI4', 0.1],
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=0.1, upper=6)
res = mp.postprocess(filename='220218.png')
mp.close()

     state  wfo   val       lon      lat                              geo  \
0       IL  ILX  12.0  -89.7800  40.4300   POINT (862987.505 -454547.319)   
1       CO  BOU  11.5 -105.2800  39.6200  POINT (-452364.928 -583606.253)   
2       CO  BOU  11.3 -105.1600  39.5100  POINT (-442822.969 -596455.378)   
3       CO  BOU  11.2 -105.2800  39.7000  POINT (-451826.563 -574732.305)   
4       CO  BOU  11.1 -105.2800  39.9300  POINT (-450275.283 -549218.304)   
...    ...  ...   ...       ...      ...                              ...   
3755    OR  MFR   0.0 -122.8820  42.3771  POINT (-1849611.611 -29985.546)   
3756    OR  MFR   0.0 -123.2655  43.0394   POINT (-1859179.314 50313.877)   
3757    OR  MFR   0.0 -121.9638  43.5491   POINT (-1743507.835 76386.805)   
3758    OR  MFR   0.0 -122.5164  42.7342     POINT (-1810393.419 365.255)   
3759    OR  MFR   0.0 -123.3572  42.9506   POINT (-1869007.446 42846.192)   

      used_for_analysis  nwsli  plotme source  xcell  ycell  
0            